In [1]:
!uv pip install -q spacy

In [2]:
!uv pip install -q nltk gensim pyLDAvis unidecode matplotlib seaborn pandas pyarrow

In [3]:
!uv pip install -q bertopic polars datasets hf_xet

In [4]:
!uv pip install -q https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-3.7.0/fr_core_news_sm-3.7.0-py3-none-any.whl

In [5]:
#compaatible uniquement avec verison python 3.11 et moins et mettre la mémoire au max
!uv pip install cudf-cu12 dask-cudf-cu12 --extra-index-url=https://pypi.nvidia.com
!uv pip install cuml-cu12 --extra-index-url=https://pypi.nvidia.com
!uv pip install cugraph-cu12 --extra-index-url=https://pypi.nvidia.com
!uv pip install cupy-cuda12x -f https://pip.cupy.dev/aarch64

Resolved 55 packages in 1.56s                                        
Prepared 33 packages in 12.06s                                           
Uninstalled 1 package in 73ms
░░░░░░░░░░░░░░░░░░░░ [0/33] Installing wheels...                                warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 33 packages in 3.95s                              
 + cachetools==5.5.2
 + cloudpickle==3.1.1
 + cuda-bindings==12.9.0
 + cuda-python==12.9.0
 + cudf-cu12==25.4.0
 + cupy-cuda12x==13.4.1
 + dask==2025.2.0
 + dask-cudf-cu12==25.4.0
 + distributed==2025.2.0
 + fastrlock==0.8.3
 + importlib-metadata==8.7.0
 + libcudf-cu12==25.4.0
 + libkvikio-cu12==25.4.0
 + librmm-cu12==25.4.0
 + locket==1.0.0
 + msgpack==1.1

In [6]:
!uv pip install -q langchain-huggingface==0.0.3

In [7]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, PartOfSpeech
from cuml.cluster import HDBSCAN
from scipy.cluster import hierarchy as sch
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from cuml.manifold import UMAP
import polars as pl
import spacy
import pandas as pd
import numpy as np
import time
from sklearn.decomposition import PCA
from langchain_huggingface import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import tqdm as notebook_tqdm

/home/onyxia/work/TopicModeling/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import html 
import json
import matplotlib.pyplot  as plt
import nltk
import numpy as np
import os
import pandas as pd
import unidecode
import re
import requests
import seaborn as sns
import string
import unidecode
import warnings
import torch
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /home/onyxia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/onyxia/nltk_data...


True

In [9]:
stop_words = set(stopwords.words('french'))

In [10]:
class CachedLemmatizer:
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.cache = {}  # Manual cache as a dictionary

    def lemmatize(self, word, pos='n'):
        if word in self.cache:
            return self.cache[word]
        else:
            lemmatized_word = self.lemmatizer.lemmatize(word, pos)
            self.cache[word] = lemmatized_word  # Store in cache
            return lemmatized_word


cached_lemmatizer = CachedLemmatizer()

In [11]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [12]:
nlp = spacy.load("fr_core_news_sm")  

In [13]:
def preprocess(docs):
    cleaned = []
    for doc in nlp.pipe(docs, batch_size=100, n_process=4):
        tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
        cleaned.append(' '.join(tokens))
    return cleaned

In [14]:
DICTIONNARY =  ['accord','entreprise', 'preambule', 'sommaire',  'code', 'syndical', 'responsable', 'representant', 
                'present', 'ca', 'organisation', 'preambule', 'peut', 'etre', 'contrat','travail', 'ressources','humaines', 'mise',
                'ainsi', 'et', 'ou', 'alors','collaborateur', 'ci', 'apres', 'party', 'signataire', 'tout', 'etat', 'cause', 'societe', 
                'notamment','article','activite', 'cette', 'donc', 'si', 'sous', 'disposition', 'convention', 'collective', 'dans', 'a', 'cadre',
                'signataire', 'partie', 'parties', 'entre', 'doit', 'mme', 'mr', 'madame', 'monsieur'
               ]

DICTIONNARY_STEM = ['part', 'signatair', 'organis', 'syndical', 
                    'dont', 'sieg', 'social', 'conseil', 'prud', 'homm', 
                   'vi', 'professionnel', 'disposit', 'legal', 'conventionnel']

In [15]:
def preprocess_text(text, lang="french"):
    # décoage HTML
    text = html.unescape(text)
    
    # nettoyage de tous les cractères spéciaux
    text = re.sub(r"&[a-z]+;", " ", text)
    text = re.sub(r"&#\d+;", " ", text)
    text = re.sub(r"[<>{}\[\]\|\^\~`\"'=]+", " ", text)
    text = re.sub(r"[–—•«»]+", " ", text)  # Tirets longs, puces, guillemets français

    # tokenisation
    words = word_tokenize(text)

    # lemming
    #stemmer = SnowballStemmer(lang)
                  
    wnl = cached_lemmatizer
   
    words_cleaned = []
    for w in words:
        #w_norm = unidecode.unidecode(w.lower())
        w_norm = w.lower()
        if (
            w_norm not in stop_words
            and w_norm not in DICTIONNARY
            and w_norm not in string.punctuation
            and not re.search(r"[<>]|--+|__+|xx+|==+", w_norm)
            and not w_norm.isnumeric()
            and len(w_norm) > 2
        ):
            words_cleaned.append(wnl.lemmatize(w_norm))
            #words_cleaned.append(stemmer.stem(w_norm))

    return words_cleaned

In [16]:
import re

def normalize(text):
    return text.lower().strip()

def split_text_by_sentences(text, flagged_sentences):
    split_texts = []
    positions = []

    normalized_text = normalize(text)

    # On garde un mapping (titre original, position) pour préserver les titres initiaux
    for sentence in flagged_sentences:
        norm_sentence = normalize(sentence)
        pos = normalized_text.find(norm_sentence)
        if pos != -1:
            # On retrouve la position réelle dans le texte original
            real_pos = text.lower().find(sentence.lower())
            if real_pos != -1:
                positions.append(real_pos)

    # Si aucune position trouvée, retourner le texte complet
    if not positions:
        return [text]

    positions = sorted(set(positions))
    positions.insert(0, 0)
    positions.append(len(text))

    for i in range(len(positions) - 1):
        start = positions[i]
        end = positions[i + 1]
        split_texts.append(text[start:end].strip())

    return split_texts



In [17]:
def split_text_with_titles(text, summary_titles):
    chunks = split_text_by_sentences(text, summary_titles)
    result = {}
    for title in summary_titles:
        for chunk in chunks:
            if normalize(title) in normalize(chunk[:len(title)+30]):
                result[title] = chunk.strip()
                break
    return result


In [18]:
model_kwargs = {'device': 'cuda'} 
#model_kwargs = {'device': 'cpu'}  
MODEL_NAME_EMBEDDER="BAAI/bge-small-en-v1.5"  #petit modèle en anglais
#MODEL_NAME_EMBEDDER="BAAI/bge-m3" #gros modèle multilingue

embedder = HuggingFaceEmbeddings(
    model_name=MODEL_NAME_EMBEDDER, 
    model_kwargs=model_kwargs,
    show_progress=True
)

phrases_non_metier = [
    "Révision de l’accord",
    "Dénonciation de l’accord",
    "Interprétation de l’accord",
    "Suivi de l’accord",
    "Durée de l’accord",
    "Formalités de publicité et de dépôt",
    "Publicité et dépôt",
    "Date d'effet et durée",
    "Champ d'application",
    "Clause de revoyure", 
    "Information des représentants du personnel", 
    "Dispositions relatives à l’accord",
    "Champ d’application",
    "Commission de suivi", 
    "Pause déjeuner du personnel", 
    "Modification de l'accord",
    "Adhésion"
    
]

# Embeddings des phrases non-métier
ref_embeddings = embedder.embed_documents(phrases_non_metier)

def filtre_par_similarite(phrases, seuil=0.85):##torp long utiliser version vectoisée
    results = []
    for phrase in phrases:
        emb = embedder.embed_query(phrase)
        sims = cosine_similarity([emb], ref_embeddings)[0]
        if max(sims) < seuil:
            results.append(phrase) 
    return results

def filtre_par_similarite_vectorise(phrases, seuil=0.85):
    if not phrases:
        return []

    phrase_embeddings = embedder.embed_documents(phrases)  
    sims = cosine_similarity(phrase_embeddings, ref_embeddings)

    # On garde les phrases dont la similarité max avec une phrase non-métier est < seuil
    keep_idx = np.max(sims, axis=1) < seuil
    return [phrase for phrase, keep in zip(phrases, keep_idx) if keep]

    
def filtre_chunks_par_titre(section_dict, phrases_non_metier, seuil=0.85): #seuil arbitraire : en tester plsr
    """
    Ne garde que les chunks dont le titre est peu similaire aux phrases non métier.
    """
    if not section_dict:
        return []

    titres = list(section_dict.keys())
    chunks = list(section_dict.values())

    # Embeddings des titres de section
    titre_embeddings = embedder.embed_documents(titres)
    ref_embeddings = embedder.embed_documents(phrases_non_metier)

    sims = cosine_similarity(titre_embeddings, ref_embeddings)

    # On garde les chunks dont le titre est peu similaire aux phrases non métier
    keep_idx = np.max(sims, axis=1) < seuil
    return [chunk.strip() for chunk, keep in zip(chunks, keep_idx) if keep]



Batches: 100%|██████████| 1/1 [00:00<00:00,  3.62it/s]


In [19]:
sommaire_hs = pd.read_parquet("data/echantillon_1000_hs_accords_TOC.parquet")
df_hs = pd.read_parquet("data/echantillon_1000_hs_accords.parquet")
df_hs = df_hs.set_index("numdossier_new")
df_hs = df_hs.merge(sommaire_hs,how="inner",left_index=True,right_index=True)
df_hs = df_hs.rename(columns={"extracted_summary":"summary"})

In [20]:
df_hs["section_dict"] = df_hs.apply(
    lambda row: split_text_with_titles(row["accorddocx"], row["summary"]),
    axis=1
)

In [21]:
def get_all_chunks(section_dict):
    chunks = list(section_dict.values())
    return [chunk.strip() for chunk in chunks]

In [22]:
def get_valid_chunks_filtered(section_dict, skip_titles=["préambule", "annexe"], seuil_sim=0.85):
    skip_titles_norm = [normalize(t) for t in skip_titles]

    # supprimer le préambule et avant 
    titles = list(section_dict.keys())
    preamble_idx = next((i for i, t in enumerate(titles) if "préambule" in normalize(t)), -1)
    if preamble_idx != -1:
        titles = titles[preamble_idx + 1:]

    # garder les titres valides uniquement
    valid_titles = [
        t for t in titles if all(skip_kw not in normalize(t) for skip_kw in skip_titles_norm)
    ]
    candidate_dict = {t: section_dict[t] for t in valid_titles}

    # filtrer par similarité des titres
    return filtre_chunks_par_titre(candidate_dict, phrases_non_metier, seuil=seuil_sim)


# Modèle en filtrant les chunks

In [23]:
df_hs["lda_documents"] = df_hs["section_dict"].apply(get_valid_chunks_filtered)

Batches: 100%|██████████| 1/1 [00:00<00:00, 50.77it/s]


In [24]:
filtered_chunks_hs = [chunk for doc_chunks in df_hs["lda_documents"] for chunk in doc_chunks]
filtered_docs_cleaned = preprocess(filtered_chunks_hs)

In [86]:
#Embeddings
start = time.time()
embedding_model = SentenceTransformer("all-MiniLM-L6-v2",device='cuda')  
embeddings = embedding_model.encode(filtered_docs_cleaned, show_progress_bar=True)
print(f"[1] Embedding en {time.time() - start:.2f}s")

Batches: 100%|██████████| 597/597 [00:26<00:00, 22.18it/s] 


[1] Embedding en 28.77s


In [26]:
print("Shape des embeddings :", embeddings_filtered.shape)


Shape des embeddings : (19074, 384)


In [95]:
#Réduction UMAP
start = time.time()
umap_model = UMAP(n_components=15, n_neighbors=30, random_state=42, metric="cosine", verbose=True)
#umap_model = UMAP(n_neighbors=, n_components=5, min_dist=0.0, metric='cosine', verbose=True)
umap_embeddings = umap_model.fit_transform(embeddings)
print(f"[2] UMAP en {time.time() - start:.2f}s")

[2025-05-16 01:26:07.613] [CUML] [info] build_algo set to brute_force_knn because random_state is given
[2025-05-16 01:26:07.624] [CUML] [debug] Computing KNN Graph
[2025-05-16 01:26:07.769] [CUML] [debug] Computing fuzzy simplicial set
[2] UMAP en 0.69s


In [28]:
#umap_embeddings_filtered.shape

In [103]:
# Clustering 
start = time.time()
#hdbscan_model =  HDBSCAN(min_cluster_size=80, metric='euclidean', cluster_selection_method='eom', prediction_data=True, verbose=True)
hdbscan_model =  HDBSCAN(min_cluster_size=80, metric='euclidean', cluster_selection_method='eom', prediction_data=True, verbose=True)

clusters = hdbscan_model.fit_predict(umap_embeddings)
print(f"[3] HDBSCAN en {time.time() - start:.2f}s")
print(f"[3] Nombre de clusters trouvés : {len(np.unique(clusters))}")

[3] HDBSCAN en 0.65s
[3] Nombre de clusters trouvés : 29


In [104]:
start = time.time()
topic_model = BERTopic(
    language="french",
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    verbose=True
)
topics, probs = topic_model.fit_transform(filtered_docs_cleaned)
print(f"[4] BERTopic final en {time.time() - start:.2f}s")


2025-05-16 01:28:02,333 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 597/597 [00:27<00:00, 21.58it/s] 
2025-05-16 01:28:30,545 - BERTopic - Embedding - Completed ✓
2025-05-16 01:28:30,546 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


[2025-05-16 01:28:30.561] [CUML] [debug] Computing KNN Graph
[2025-05-16 01:28:30.649] [CUML] [debug] Computing fuzzy simplicial set


2025-05-16 01:28:31,272 - BERTopic - Dimensionality - Completed ✓
2025-05-16 01:28:31,274 - BERTopic - Cluster - Start clustering the reduced embeddings


[2025-05-16 01:28:31.131] [CUML] [debug] Running transform
[2025-05-16 01:28:31.131] [CUML] [debug] Building KNN Graph
[2025-05-16 01:28:31.226] [CUML] [debug] Smoothing KNN distances
[2025-05-16 01:28:31.227] [CUML] [debug] Executing fuzzy simplicial set
[2025-05-16 01:28:31.228] [CUML] [debug] Performing L1 normalization
[2025-05-16 01:28:31.237] [CUML] [debug] n_epochs=30
[2025-05-16 01:28:31.240] [CUML] [debug] Computing # of epochs for training each sample
[2025-05-16 01:28:31.240] [CUML] [debug] Performing optimization


2025-05-16 01:28:31,847 - BERTopic - Cluster - Completed ✓
2025-05-16 01:28:31,854 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-16 01:28:32,924 - BERTopic - Representation - Completed ✓


[4] BERTopic final en 31.01s


In [105]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,11006,-1_travail_heure_temps_salarié,"[travail, heure, temps, salarié, jour, article...",[article 3.2 2 principe temps partiel moduler ...
1,0,2425,0_heure_supplémentaire_jour_repos,"[heure, supplémentaire, jour, repos, congé, an...",[article 4 solde fin période référence heure s...
2,1,948,1_rémunération_absence_période_salarié,"[rémunération, absence, période, salarié, base...",[b incidence absence cours période \n\n cas pé...
3,2,764,2_article_chapitre_jour_forfait,"[article, chapitre, jour, forfait, duree, trav...","[article 2, article 4, article 5]"
4,3,563,3_révision_accord_party_dénonciation,"[révision, accord, party, dénonciation, présen...",[denonciation revision \n\n présent accord pou...
5,4,414,4_temps_pause_effectif_travail,"[temps, pause, effectif, travail, vaquer, occu...",[3.1 durée effectif travail \n application cod...
6,5,352,5_exemplaire_déposer_accord_dépôt,"[exemplaire, déposer, accord, dépôt, prud, con...",[article 7 dépôt publicité accord \n présent a...
7,6,290,6_temps_article_aménagement_amenagement,"[temps, article, aménagement, amenagement, tra...",[article 5 aménagement temps travail temps par...
8,7,260,7_vigueur_accord_indéterminé_conclure,"[vigueur, accord, indéterminé, conclure, durée...",[4 1 durée entrée vigueur accord \n\n présen...
9,8,254,8_déconnexion_télétravail_outil_droit,"[déconnexion, télétravail, outil, droit, profe...",[article 4 droit déconnexion \n\n droit déco...


In [106]:
fig = topic_model.visualize_barchart()
fig.write_html("barchart_topics_bis.html")

In [107]:
fig = topic_model.visualize_topics()
fig.write_html("visualize_topics_bis.html")

# Modèle sur le topic principal 

* On veut rediviser le topic 0
* Pas mal de topic qui sont du bruit

--> Relancer un Bertopic sur le topic 0 uniquement

In [1]:
# Topic principal est le topic 0
topic_id = 0

# 1. Récupérer les indices des documents de ce topic
docs = topic_model.get_document_info(filtered_docs_cleaned)
docs_in_big_topic = docs[docs['Topic'] == topic_id]

# 2. Extraire les textes concernés
sub_docs = docs_in_big_topic['Document'].tolist()

# 3. Recalculer les embeddings (si tu veux affiner)
from sentence_transformers import SentenceTransformer
embedding_model_sub = SentenceTransformer("all-MiniLM-L6-v2")
sub_embeddings = embedding_model.encode(sub_docs, show_progress_bar=True)

#Réduction UMAP
start = time.time()
umap_model_sub = UMAP(n_components=15, n_neighbors=15, random_state=42, metric="cosine", verbose=True)
#umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', verbose=True)
umap_embeddings_filtered_sub = umap_model.fit_transform(sub_embeddings)
print(f"[2] UMAP en {time.time() - start:.2f}s")


# Clustering 
start = time.time()
#hdbscan_model =  HDBSCAN(min_cluster_size=80, metric='euclidean', cluster_selection_method='eom', prediction_data=True, verbose=True)
hdbscan_model_sub =  HDBSCAN(min_cluster_size=100, metric='euclidean', cluster_selection_method='eom', prediction_data=True, verbose=True)
clusters = hdbscan_model.fit_predict(umap_embeddings_filtered_sub)
print(f"[3] HDBSCAN en {time.time() - start:.2f}s")
print(f"[3] Nombre de clusters trouvés : {len(np.unique(clusters))}")

# 4. Créer un nouveau BERTopic sur ces sous-documents

sub_model = BERTopic(
    language="french",
    embedding_model=embedding_model_sub,
    umap_model=umap_model_sub,
    hdbscan_model=hdbscan_model,
    verbose=True
)
sub_model.fit(sub_docs, embeddings=sub_embeddings)

# 5. Visualiser les nouveaux sous-topics
fig = sub_model.visualize_topics()
fig.write_html("sub_visualize_topics.html")


NameError: name 'topic_model_filtered' is not defined

In [110]:
fig_sub = sub_model.visualize_barchart()
fig_sub.write_html("sub_visualize_barchart.html")

# KeyBERTInspired

In [34]:
representation_model = KeyBERTInspired()

topic_model_KeyBERTInspired = BERTopic(representation_model=representation_model,
                                       embedding_model=embedding_model,
                                       umap_model=umap_model,
                                       hdbscan_model=hdbscan_model,
                                       verbose=True,
                                       language="french")
topics_KeyBERTInspired, probs_KeyBERTInspired = topic_model_KeyBERTInspired.fit_transform(filtered_docs_cleaned)

2025-05-16 00:23:01,319 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 597/597 [00:28<00:00, 21.24it/s] 
2025-05-16 00:23:30,207 - BERTopic - Embedding - Completed ✓
2025-05-16 00:23:30,209 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


[2025-05-16 00:23:30.229] [CUML] [debug] Computing KNN Graph
[2025-05-16 00:23:30.322] [CUML] [debug] Computing fuzzy simplicial set


2025-05-16 00:23:31,246 - BERTopic - Dimensionality - Completed ✓
2025-05-16 00:23:31,250 - BERTopic - Cluster - Start clustering the reduced embeddings


[2025-05-16 00:23:31.086] [CUML] [debug] Running transform
[2025-05-16 00:23:31.086] [CUML] [debug] Building KNN Graph
[2025-05-16 00:23:31.186] [CUML] [debug] Smoothing KNN distances
[2025-05-16 00:23:31.188] [CUML] [debug] Executing fuzzy simplicial set
[2025-05-16 00:23:31.190] [CUML] [debug] Performing L1 normalization
[2025-05-16 00:23:31.206] [CUML] [debug] n_epochs=30
[2025-05-16 00:23:31.212] [CUML] [debug] Computing # of epochs for training each sample
[2025-05-16 00:23:31.212] [CUML] [debug] Performing optimization


2025-05-16 00:23:31,961 - BERTopic - Cluster - Completed ✓
2025-05-16 00:23:31,972 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-16 00:23:33,657 - BERTopic - Representation - Completed ✓


In [35]:
len(filtered_docs_cleaned)

19074

In [36]:
topic_model_KeyBERTInspired.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,549,-1_entreprendre_salarier_demande_salarié,"[entreprendre, salarier, demande, salarié, rém...",[article 3.1 compartiment distribution distrib...
1,0,18385,0_salarier_salarié_demande_entreprendre,"[salarier, salarié, demande, entreprendre, rém...",[durée journalier travail temps travail effect...
2,1,76,1_accord_entreprise_entreprendre_organisation,"[accord, entreprise, entreprendre, organisatio...","[accord entreprise, accord entreprise, accord ..."
3,2,23,2_preambule___,"[preambule, , , , , , , , , ]","[preambule, preambule, preambule ]"
4,3,21,3_réserve_adaptation_déplacement_salarié,"[réserve, adaptation, déplacement, salarié, pr...",[article 3 1 salarié concerner \n ouvrier no...
5,4,20,4_deplacement_déplacement_petit_petits,"[deplacement, déplacement, petit, petits, arti...","[article 2 petit déplacement, article 3 pe..."


In [37]:
topic_model_KeyBERTInspired.visualize_topics()

In [38]:
topic_model_KeyBERTInspired.visualize_barchart()

# BERTopic (MMR)

In [39]:
representation_model = MaximalMarginalRelevance(diversity=0.3)

topic_model_MMR = BERTopic(representation_model=representation_model,
                                       embedding_model=embedding_model,
                                       umap_model=umap_model,
                                       hdbscan_model=hdbscan_model,
                                       verbose=True,
                                       language="french")
topics_MMR, probs_MMR = topic_model_MMR.fit_transform(filtered_docs_cleaned)

2025-05-16 00:23:34,127 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 597/597 [00:28<00:00, 20.80it/s] 
2025-05-16 00:24:03,865 - BERTopic - Embedding - Completed ✓
2025-05-16 00:24:03,866 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-16 00:24:04,876 - BERTopic - Dimensionality - Completed ✓
2025-05-16 00:24:04,879 - BERTopic - Cluster - Start clustering the reduced embeddings


[2025-05-16 00:24:03.885] [CUML] [debug] Computing KNN Graph
[2025-05-16 00:24:03.979] [CUML] [debug] Computing fuzzy simplicial set
[2025-05-16 00:24:04.704] [CUML] [debug] Running transform
[2025-05-16 00:24:04.704] [CUML] [debug] Building KNN Graph
[2025-05-16 00:24:04.814] [CUML] [debug] Smoothing KNN distances
[2025-05-16 00:24:04.816] [CUML] [debug] Executing fuzzy simplicial set
[2025-05-16 00:24:04.818] [CUML] [debug] Performing L1 normalization
[2025-05-16 00:24:04.834] [CUML] [debug] n_epochs=30
[2025-05-16 00:24:04.839] [CUML] [debug] Computing # of epochs for training each sample
[2025-05-16 00:24:04.840] [CUML] [debug] Performing optimization


2025-05-16 00:24:05,427 - BERTopic - Cluster - Completed ✓
2025-05-16 00:24:05,437 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-16 00:24:08,781 - BERTopic - Representation - Completed ✓


In [40]:
topic_model_MMR.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,11249,-1_travail_heure_jour_salarié,"[travail, heure, jour, salarié, accord, supplé...",[article 8.3 congé payer \t 18 \n 8.3.1 défi...
1,0,520,0_révision_signataire_accord_recommander,"[révision, signataire, accord, recommander, 22...",[article 5 dénonciation révision \n\n présent...
2,1,374,1_pause_temps_occupation_travail,"[pause, temps, occupation, travail, directive,...",[2 1 temps travail effectif \n rappeler temps ...
3,2,313,2_temps_amenagement_travail_effectif,"[temps, amenagement, travail, effectif, modula...","[article 2 temps travail \t 6, article 5 aména..."
4,3,296,3_absence_rémunération_rémunérer_rupture,"[absence, rémunération, rémunérer, rupture, sa...",[4.5 absence arriver départ cours période \n\n...
...,...,...,...,...,...
74,73,22,73_déconnexion_deconnexion_droit_sensibilisation,"[déconnexion, deconnexion, droit, sensibilisat...","[droit déconnexion, 4.4.2 \t droit déconnexion..."
75,74,21,74_réglementaire_imputation_supplémentaire_con...,"[réglementaire, imputation, supplémentaire, co...",[4.1 définition \n\n constituer heure supplé...
76,75,21,75_sédentaire_bâtiment_apporter_adaptation,"[sédentaire, bâtiment, apporter, adaptation, d...",[article 3 1 salarié concerner \n ouvrier no...
77,76,21,76_consécutif_3121_hebdomadaire_minimal,"[consécutif, 3121, hebdomadaire, minimal, ampl...",[4.3 amplitude repos \n rappeler compte tenir ...


 # Hierarchical topics 

In [41]:
hierarchical_topics = topic_model_MMR.hierarchical_topics(filtered_docs_cleaned)
hierarchical_topics

100%|██████████| 77/77 [00:02<00:00, 36.35it/s]


,Parent_ID,Parent_Name,Topics,Child_Left_ID,Child_Left_Name,Child_Right_ID,Child_Right_Name,Distance
76,154,travail_accord_salarié_repos_disposition,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",151,révision_dépôt_société_signataire_dénonciation,153,travail_jour_article_rémunération_absence,2.039367
75,153,travail_jour_article_rémunération_absence,"[1, 3, 6, 7, 8, 9, 10, 11, 14, 15, 16, 18, 19,...",152,jour_travail_salarié_période_absence,145,supplémentaire_heure_contingent_complémentaire...,1.840023
74,152,jour_travail_salarié_période_absence,"[1, 3, 6, 7, 8, 9, 10, 15, 16, 19, 22, 25, 26,...",130,hebdomadaire_amplitude_travail_consécutiver_jo...,150,jour_salarié_travail_période_absence,1.692324
73,151,révision_dépôt_société_signataire_dénonciation,"[0, 2, 4, 5, 12, 13, 17, 21, 23, 24, 27, 34, 3...",135,trajet_zone_kilomètre_déplacement_petit,149,société_signataire_dénonciation_dépôt_disposition,1.459704
72,150,jour_salarié_travail_période_absence,"[1, 3, 6, 7, 8, 9, 10, 15, 16, 19, 25, 26, 28,...",122,contrepartie_obligatoire_compensateur_demande_...,148,jour_salarié_temps_travail_absence,1.454840
...,...,...,...,...,...,...,...,...
4,82,partiel_annualisation_temps_salarier_travail,"[15, 47]",15,annualisation_temps_période_plein_salarié,47,partiel_salarier_hebdomadaire_salarié_période,0.299154
3,81,repos_hebdomadaire_consécutif_consécutiver_min...,"[30, 57]",30,repos_hebdomadaire_consécutiver_consécutif_ajo...,57,quotidien_repos_article11_hebdomadaires_minimal,0.295648
2,80,contingent_supplémentaire_heure_220_contrepartie,"[11, 32, 54]",32,contingent_supplementaire_heure_fixer_dépassement,79,contingent_supplémentaire_heure_annuel_etam,0.287376
1,79,contingent_supplémentaire_heure_annuel_etam,"[11, 54]",11,contingent_supplémentaire_heure_annuel_etam,54,contingent_supplémentaire_accomplie_obligatoir...,0.154028


In [42]:
linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)
hierarchical_topics = topic_model_MMR.hierarchical_topics(filtered_docs_cleaned, linkage_function=linkage_function)

100%|██████████| 77/77 [00:02<00:00, 26.81it/s]


In [43]:
topic_model_MMR.visualize_hierarchy(hierarchical_topics=hierarchical_topics)